In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
# Ignore all warnings
warnings.filterwarnings("ignore")

/Users/prayer/anaconda3/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [4]:
import pandas as pd

train = pd.read_csv('train.csv', index_col='Id')
X_test = pd.read_csv('test.csv', index_col='Id')

# Separate X_train and y_train
X_train = train.drop('SalePrice', axis=1)
y_train = train['SalePrice']

# Separate numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=['number']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns


In [5]:
# Assume you have X_train, X_cv, X_test, y_train, numerical_cols, and categorical_cols defined

# Fill missing values
X_train[numerical_cols] = X_train[numerical_cols].fillna(X_train[numerical_cols].median())
X_train[categorical_cols] = X_train[categorical_cols].fillna('miss')

X_test[numerical_cols] = X_test[numerical_cols].fillna(X_test[numerical_cols].median())
X_test[categorical_cols] = X_test[categorical_cols].fillna('miss')

X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



In [6]:
# Normalize numerical columns
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_cv[numerical_cols] = scaler.transform(X_cv[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])




In [7]:
# One-hot encode categorical columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[categorical_cols])
X_cv_encoded = encoder.transform(X_cv[categorical_cols])
X_test_encoded = encoder.transform(X_test[categorical_cols])

In [8]:
# Get the column names for the one-hot encoded features
encoded_feature_names = encoder.get_feature_names_out(categorical_cols)

# Create DataFrames from the encoded arrays
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoded_feature_names,index = X_train.index)
X_cv_encoded_df = pd.DataFrame(X_cv_encoded, columns=encoded_feature_names,index = X_cv.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoded_feature_names,index = X_test.index)

In [9]:
# Concatenate the encoded categorical columns with the numerical columns
X_train = pd.concat([X_train_encoded_df, X_train[numerical_cols]],axis=1)
X_cv = pd.concat([X_cv_encoded_df, X_cv[numerical_cols]], axis=1)
X_test = pd.concat([X_test_encoded_df, X_test[numerical_cols]] ,axis=1)


In [ ]:
X_test

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

# List of algorithms with hyperparameters for tuning
algorithms = [
    (LinearRegression(), {}),
    (DecisionTreeRegressor(), {'max_depth': [3, 5, 7]}),
    (RandomForestRegressor(), {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}),
    (GradientBoostingRegressor(), {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]}),
    (SVR(), {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}),
    (KNeighborsRegressor(), {'n_neighbors': [3, 5, 7]}),
    (Ridge(), {'alpha': [0.1, 1, 10]}),
    (Lasso(), {'alpha': [0.1, 1, 10]}),
    (ElasticNet(), {'alpha': [0.1, 1, 10], 'l1_ratio': [0.1, 0.5, 0.9]}),
    (AdaBoostRegressor(), {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2]}),
    (ExtraTreesRegressor(), {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}),
    (GaussianProcessRegressor(), {}),
    (KernelRidge(), {'alpha': [0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf']}),
    (MLPRegressor(), {'hidden_layer_sizes': [(50,), (100,), (150,)], 'activation': ['relu', 'tanh']}),
    (XGBRegressor(), {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]}),
]
# Loop through each algorithm
for algo, param_grid in algorithms:
    grid_search = GridSearchCV(estimator=algo, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.best_estimator_.predict(X_cv)
    r2 = r2_score(y_cv, y_pred)
    print(f"{algo.__class__.__name__}: Best Parameters - {grid_search.best_params_}, Best R2 Score - {r2}")


In [ ]:
from tqdm import tqdm

# List of algorithms with hyperparameters for tuning
algorithms = [
   
#     (GradientBoostingRegressor(), {'n_estimators': [50,70, 100,120, 150,180,200,250,350], 'learning_rate': [0.01,.05, 0.09,.1,.13,.19, 0.2,.25,.3], 'max_depth': [1,2,3,4 ,5, 7,10,15]}),
#     (KernelRidge(), {'alpha': [.001,.05,.075,0.1,.15,.2,.3,.6, 1,2,5, 10], 'kernel': ['linear', 'poly', 'rbf']}),
    (XGBRegressor(), {'n_estimators': [50, 150,200,250,300,325,350,375,400,450,500,600], 'learning_rate': [0.01,.05 ,0.09,.1,.125,.13,.15,.17,.19, 0.2,.3], 'max_depth': [1,2,3,4 ,5, 7,10,15]}),
]


# Loop through each algorithm
for algo, param_grid in tqdm(algorithms):
    grid_search = GridSearchCV(estimator=algo, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.best_estimator_.predict(X_cv)
    r2 = r2_score(y_cv, y_pred)
    print(f"{algo.__class__.__name__}: Best Parameters - {grid_search.best_params_}, Best R2 Score - {r2}")

  0%|                                                     | 0/1 [00:00<?, ?it/s]

In [ ]:
model = XGBRegressor(learning_rate= 0.13, max_depth= 3, n_estimators= 350)
model.fit(X_train, y_train)
y_pred = model.predict(X_cv)
r2 = r2_score(y_cv, y_pred)
SalePrice = model.predict(X_test)

In [ ]:
r2

In [ ]:
submission = pd.DataFrame()
submission['id'] = X_test.index
submission['SalePrice'] = SalePrice
submission

In [ ]:
# Assuming submission is your DataFrame
submission.to_csv('submission.csv', index=False)
